In [ ]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from get_data import get_historical_from_db
import ccxt
plt.style.use("fivethirtyeight")

In [ ]:
df = get_historical_from_db(
    ccxt.binance(), 
    "MATIC/USDT",
    "1h",
    path="/home/Astra_world/"
)

On s'entraine à normaliser l'ensemble des données

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1)).fit_transform(df.filter(["close"]).values)

On récupère les prix de fermeture de 2020

In [ ]:
data=df[:"2020"].filter(["close"])
dataset=data.values
training_data_len=math.ceil(len(dataset))

In [ ]:
train_data=scaled_data[0:training_data_len,:]
x_train,y_train=[],[]
for i in range (7,len(train_data)):
    x_train.append(train_data[i-7:i,0])
    y_train.append(train_data[i,0])

In [ ]:
x_train,y_train=np.array(x_train,dtype=object).astype(np.float32),np.array(y_train,dtype=object).astype(np.float32)

In [ ]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(LSTM(units = 50, return_sequences = False))
regressor.add(Dense(25))
regressor.add(Dense(1))

In [ ]:
regressor.compile(optimizer="adam",loss="mean_squared_error")

In [ ]:
regressor.fit(x_train,y_train,batch_size=1,epochs=1)

In [ ]:
data=df["2022"].filter(["close"])
dataset=data.values
training_data_len=math.ceil(len(dataset))

In [ ]:
scaled_data=scaler.fit_transform(dataset)

In [ ]:
train_data=scaled_data[0:training_data_len,:]
x_train,y_train=[],[]
for i in range (7,len(train_data)):
    x_train.append(train_data[i-7:i,0])
    y_train.append(train_data[i,0])

In [ ]:
x_train,y_train=np.array(x_train,dtype=object).astype(np.float32),np.array(y_train,dtype=object).astype(np.float32)

In [ ]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [ ]:
predictions=regressor.predict(x_train)
predictions=scaler.inverse_transform(predictions)

In [ ]:
rmse=np.sqrt(np.mean(predictions-y_train)**2)
rmse